In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_loader import *
from  AL_strategies import *
from models import *
from metrics import *
from Stream_Bootstrap_helpers import *

In [ ]:
## Load dataset
label_list = ['Sad or depressed', 'Calm or peaceful', 'Stressed or anxious', 'sleep_class', 'Angry or frustrated']
output_folder = 'Dataset/FnF'

# Load the data for each label
data_dict = load_X_y_graphs(dataset='FnF, output_folder=Dataset/FnF')

# Retrieving data for "Sad or depressed"
X_sad = data_dict['Sad or depressed']['X']
y_sad = data_dict['Sad or depressed']['y']
L_friend_sad = data_dict['Sad or depressed']['L_friend']

# Retrieving data for "Stressed or anxious"
X_stress = data_dict['Stressed or anxious']['X']
y_stress = data_dict['Stressed or anxious']['y']
L_friend_stress = data_dict['Stressed or anxious']['L_friend']

# Retrieving data for "sleep_class"
X_sleep = data_dict['sleep_class']['X']
y_sleep = data_dict['sleep_class']['y']
L_friend_sleep = data_dict['sleep_class']['L_friend']

# Retrieving data for "Angry or frustrated"
X_angry = data_dict['Angry or frustrated']['X']
y_angry = data_dict['Angry or frustrated']['y']
L_friend_angry = data_dict['Angry or frustrated']['L_friend']

# Retrieving data for "Calm or peaceful"
X_calm = data_dict['Calm or peaceful']['X']
y_calm = data_dict['Calm or peaceful']['y']
L_friend_calm = data_dict['Calm or peaceful']['L_friend']


In [ ]:
## A function to run all AL stratgies and save the results 
def evaluate_and_plot_all_strategies_with_uncertainty(X, y, A, num_days, num_nodes, num_features, L, k, model_class,
                                                      metrics_list, n_bootstraps, dynamic_A, num_classes, train_mask,
                                                      test_mask, seed=None, save_folder="results", model_params=None):
    all_metrics_df = []  # To collect metrics for all strategies

    # Set up a folder for consolidated metrics and user logs
    metrics_folder = os.path.join(save_folder, "metrics")
    os.makedirs(metrics_folder, exist_ok=True)

    # Define strategies including each uncertainty variant and the 'no_AL' strategy
    strategies = ['graphpart', 'graphpartfar', 'age', 'coreset', 'featProp', "density",
                  "uncertainty_entropy", "uncertainty_least_confidence", "uncertainty_margin",
                  "pagerank", "degree", "random_sample", "no_AL"]

    # Map uncertainty strategy names to the actual metric used in uncertainty_strategy
    uncertainty_strategies = {
        "uncertainty_entropy": "entropy",
        "uncertainty_least_confidence": "least_confidence",
        "uncertainty_margin": "margin"
    }

    for strategy in strategies:
        print(f"Running bootstrap and evaluation for strategy: {strategy}")
        
        # Adjust strategy and k for 'no_AL'
        if strategy == 'no_AL':
            current_strategy =  "random_sample"
            current_k = 0
        else:
            current_strategy = strategy
            current_k = k

        # Determine if the current strategy is an uncertainty-based one
        if current_strategy.startswith("uncertainty"):
            # Set the uncertainty metric from the mapping
            uncertainty_metric = uncertainty_strategies[current_strategy]
            metrics_df = bootstrap_and_evaluate(
                X=X, y=y, A=A, num_days=num_days, num_nodes=num_nodes, num_features=num_features,
                L=L, k=current_k, model_class=model_class, strategy="uncertainty", uncertainity_metric=uncertainty_metric,
                metrics_list=metrics_list, n_bootstraps=n_bootstraps, dynamic_A=dynamic_A,
                num_classes=num_classes, train_mask=train_mask, test_mask=test_mask, seed=seed,
                save_folder=save_folder, model_params=model_params  # Pass model_params here
            )
        else:
            # For non-uncertainty strategies including 'no_AL' which uses 'random'
            metrics_df = bootstrap_and_evaluate(
                X=X, y=y, A=A, num_days=num_days, num_nodes=num_nodes, num_features=num_features,
                L=L, k=current_k, model_class=model_class, strategy=current_strategy, uncertainity_metric=None,
                metrics_list=metrics_list, n_bootstraps=n_bootstraps, dynamic_A=dynamic_A,
                num_classes=num_classes, train_mask=train_mask, test_mask=test_mask, seed=seed,
                save_folder=save_folder, model_params=model_params  # Pass model_params here
            )

        # Add strategy as a column in the metrics DataFrame
        metrics_df["strategy"] = strategy  # Keep original strategy name for clarity
        all_metrics_df.append(metrics_df)

    # Combine all DataFrames
    consolidated_df = pd.concat(all_metrics_df, ignore_index=True)

    # Save the consolidated metrics DataFrame for all strategies in the metrics folder
    consolidated_metrics_path = os.path.join(metrics_folder, "consolidated_metrics_all_strategies_with_baselines.csv")
    consolidated_df.to_csv(consolidated_metrics_path, index=False)


In [ ]:

# Define label data structure
data_dict = {
    'Calm or peaceful': {'X': X_calm, 'y': y_calm, 'L_friend': L_friend_calm},
    # 'Sad or depressed': {'X': X_sad, 'y': y_sad, 'L_friend': L_friend_sad},
    # 'Stressed or anxious': {'X': X_stress, 'y': y_stress, 'L_friend': L_friend_stress},
    # 'Angry or frustrated': {'X': X_angry, 'y': y_angry, 'L_friend': L_friend_angry},

}

# Define graph types
graph_types = ['L_friend', 'L_duration', 'L_non_zero', 'L_couple']

# Define values for L and k to iterate over
L_values = [2,4,6,8,10]  # Example values for L
k_values = [2, 4, 6,8, 10]    # Example values for k

# Model and experiment parameters
num_classes = 2
metrics_list = ['accuracy', 'precision', 'recall', 'roc_auc', 'pr_auc', 'f1_micro', 'f1_macro']
n_bootstraps = 100
dynamic_A = False
train_ratio = 0.8
model_class = GATModel
model_params = {
    'hidden_channels': 64,
    'num_heads': 8,
    'num_layers': 2,
    'dropout': 0.7,
    'activation': "relu",
    'batchnorm': True
}
seed = 42

# Main folder for results
base_folder = "Results_FnF"
os.makedirs(base_folder, exist_ok=True)

# Run experiments for each label, graph type, and (L, k) combination
for label, data in data_dict.items():
    label_folder = os.path.join(base_folder, label)
    os.makedirs(label_folder, exist_ok=True)
    
    for graph_type in graph_types:
        graph_folder = os.path.join(label_folder, graph_type)
        os.makedirs(graph_folder, exist_ok=True)
        
        
        for L in L_values:
            for k in k_values:
                subfolder_name = f"L_{L}_k_{k}"
                experiment_folder = os.path.join(graph_folder, subfolder_name)
                os.makedirs(experiment_folder, exist_ok=True)
                # Print the current configuration being processed
                print(f"Processing Label: {label}, Graph: {graph_type}, L: {L}, k: {k}")
                
                
                # Prepare inputs for the current label and graph type
                X = data['X']
                y = data['y']
                A = data.get(graph_type, None)
                
                if A is None:
                    continue  # Skip if the graph type is not found in the data
                
                num_days, num_nodes, num_features = X.shape
                
                # Generate train and test masks
                train_mask, test_mask = generate_train_test_masks(num_nodes, train_ratio=train_ratio)
                train_mask = torch.tensor(train_mask, dtype=torch.bool) if isinstance(train_mask, np.ndarray) else train_mask
                test_mask = torch.tensor(test_mask, dtype=torch.bool) if isinstance(test_mask, np.ndarray) else test_mask
                
                # Run experiment
                evaluate_and_plot_all_strategies_with_uncertainty(
                    X, y, A, num_days, num_nodes, num_features, L, k, model_class,
                    metrics_list, n_bootstraps, dynamic_A, num_classes, train_mask,
                    test_mask, seed=seed, save_folder=experiment_folder, model_params=model_params
                )

                # Free up memory after each iteration
                del X, y, A, train_mask, test_mask
               

print("All experiments completed and results saved.")
